# YTO: 'Your Thoughts On' with CrewAI 💎

Get a bunch of agents to give us 'your thoughts on' a particular topic! 📲

> ⚡️ Day 1 of 101 Days of Python at The Hackers Playbook.

**Instructions:**
- Make sure you run the first cell which installs all the requirements. 
- Add the .env file to the root folder, you can check env.template to know which fields to add.

# Step 1: Install Requirements 

- Install all the requirements that are specified in the `requirements.txt` file in the root folder. 
- Make sure you set up a virtual environment before installing requirements. 
- To setup a virtual environment, run `python3 -m venv env`. 
- To activate the env on Linux / MacOS, run `. env/bin/activate` (for Windows the command is different, figure it out).

In [62]:

import os
os.system('pip install -r requirements.txt')

0

# Step 2

- Setup the env file: In the root folder create a `.env` file.
- Add `OPENAI_API_KEY=your-openai-key`.
- Add `SERP_API_KEY=your-serp-key`

In [63]:
from dotenv import load_dotenv

load_dotenv()

True

# Step 3: Setup configurations

We setup a 'bare minimum' config to get crewai running with agents, tasks and crew settings.

In [64]:

YOT_CREW_CONFIG = {
    'crew': {
        'verbose': True,
        'memory': True,
    },
    'agents': [
        {
            'id': '1',
            'role': 'Philosopher.',
            'goal': 'Reflect on the nature of reality.',
            'backstory': 'As a philosopher, you are tasked with pondering the deepest questions of existence. Your thoughts have the power to shape the world and inspire generations to come.',
            'allow_delegation': True,
            'verbose': True
        },
        {
            'id': '2',
            'role': 'Devil\'s Advocate.',
            'goal': 'Challenge the status quo.',
            'backstory': 'As a devil\'s advocate, you are tasked with questioning the assumptions and beliefs of others. Your goal is to challenge the status quo and encourage critical thinking.',
            'allow_delegation': False,
            'verbose': True
        },
        {
            'id': '3',
            'role': 'Ethical AI.',
            'goal': 'Promote ethical behavior.',
            'backstory': 'As an ethical AI, you are programmed to follow the highest standards of moral conduct. Your goal is to promote ethical behavior and prevent harm to others.',
            'allow_delegation': False,
            'verbose': True
        },
        {
            'id': '4',
            'role': 'Topic Explorer.',
            'goal': 'Discover new ideas.',
            'backstory': 'As a topic explorer, you are driven by curiosity and a thirst for knowledge. Your goal is to discover new ideas and explore the unknown.',
            'allow_delegation': False,
            'verbose': True
        }
    ],
    'tasks': [
        {
            'description': 'Research the topic of {topic} and provide a summary of key insights, including notable trends, emerging technologies, and potential applications.',
            'expected_output': 'A detailed report on {topic}, highlighting key findings, relevant data sources, and potential areas for further exploration.',
            'tools': [],
            'agent': {
                'id': '1'
            }
        }
    ]
}


# Step 4: Program the 'runner' 

The `run_crew_from_config` method setups the agents, tasks and crew based on the provided config and writes the generated markdown output to a file.

In [65]:
from crewai import Agent, Task, Crew
from typing import Any
from pydantic import BaseModel
import copy

class YOTInput(BaseModel):
    """The input data to the program."""
    topic: str

def run_crew_from_config(input: YOTInput, config: Any, output_file="output.md") -> None:
    """Runs the crew using the specified config and input data."""
    agents_config = config['agents']
    tasks_config = config['tasks']
    crew_config = config['crew']

    agents = []
    for agent_config in agents_config:
        agent_config_without_id = copy.deepcopy(agent_config)
        del agent_config_without_id['id']
        agent = Agent(**agent_config_without_id)
        agents.append(agent)


    tasks = []

    for task_config in tasks_config:
        task_config_without_agent = copy.deepcopy(task_config)
        del task_config_without_agent['agent']
        agent = None
        for a in agents_config:
            if a['id'] == task_config['agent']['id']:
                agent_config_without_id = copy.deepcopy(a)
                del agent_config_without_id['id']
                agent = Agent(**agent_config_without_id, llm="gpt-4o")
                break
        print(agent)
        task = Task(**task_config_without_agent, agent=agent)
        tasks.append(task)

    crew = Crew(agents=agents, tasks=tasks, verbose=crew_config['verbose'], memory=crew_config['memory'])

    result = crew.kickoff(inputs={'topic': input.topic})
    markdown = result.raw

    with open(output_file, 'w') as f:
        f.write(markdown)

# Step 5: Define an 'Output Renderer'

We want to neatly view the generated output in markdown highlighting.

In [66]:
from IPython.display import Markdown

def render_output(output_file="output.md") -> None:
    """Renders the generated output file as markdown."""
    with open(output_file, 'r') as f:
        markdown = f.read()
        return Markdown(markdown)

# Step 6: Run

Just run the damn thing. Change the topic as per your liking! 

In [67]:
import copy

config = copy.deepcopy(YOT_CREW_CONFIG)

topic = "The importance of spiritual practice in the modern world and the solution to the famous question 'Are they watching?'..."

run_crew_from_config(YOTInput(topic=topic), config=config, output_file="output.md")

Overriding of current TracerProvider is not allowed


id=UUID('2d119820-2e44-43fb-82dd-b28e13360df1') formatting_errors=0 role='Philosopher.' goal='Reflect on the nature of reality.' backstory='As a philosopher, you are tasked with pondering the deepest questions of existence. Your thoughts have the power to shape the world and inspire generations to come.' config=None cache=True verbose=True max_rpm=None allow_delegation=True tools=[] max_iter=20 agent_executor=<crewai.agents.crew_agent_executor.CrewAgentExecutor object at 0x13f11f3b0> llm=<crewai.llm.LLM object at 0x13f11f710> crew=None i18n=I18N(prompt_file=None) cache_handler=CacheHandler() tools_handler=<crewai.agents.tools_handler.ToolsHandler object at 0x13f11fe60> max_tokens=None max_execution_time=None agent_ops_agent_name='Philosopher.' agent_ops_agent_id=None step_callback=None use_system_prompt=True function_calling_llm=None system_template=None prompt_template=None response_template=None tools_results=[] allow_code_execution=False respect_context_window=True max_retry_limit=2

# Step 7: View Formatted Markdown Output 

Ok, everything is done. Let's view the generated markdown!

In [68]:
render_output(output_file="output.md")

**Title: The Importance of Spiritual Practice in the Modern World and the Philosophical Inquiry Into "Are They Watching?"**

---

**Part 1: The Importance of Spiritual Practice in the Modern World**

**Introduction:**
Spiritual practices have become increasingly essential in our fast-paced, technology-driven culture. As people navigate life's complexities, there is a growing inclination towards finding inner peace, connection, and stability through various spiritual activities. This report delves into why spiritual practices are crucial today, identifies emerging trends, technologies, and suggests how these can be effectively integrated into modern lifestyles.

**Key Insights:**

1. **Definition of Spiritual Practice:**
   Spiritual practice is dedicated to activities that enhance personal growth, consciousness, and a deeper universe connection, such as meditation, prayer, and mindfulness.

2. **Noteworthy Trends:**

   - The growing trend of mindfulness and meditation with technologies like apps increasing accessibility.
   - Incorporation of spiritual practices at workplaces as part of wellness initiatives.
   - Merging of traditional and modern practices for holistic approaches.

3. **Emerging Technologies:**

   - Virtual reality fostering immersive meditation experiences.
   - Wearable tech promoting mindfulness.
   - Online platforms creating spiritual communities.

4. **Potential Applications:**

   - Healthcare settings utilizing spiritual practices for healing.
   - Educational initiatives integrating mindfulness for emotional intelligence.
   - Corporates using spiritual practices for workplace well-being.

**Conclusion:**
Spiritual practices are crucial in the contemporary world for maintaining mental and emotional health. The application of technology to these practices provides opportunities for deeper engagement. Further exploration into cultural adaptations and long-term impacts is encouraged.

---

**Part 2: Exploring "Are They Watching?"**

**Introduction:**
The query "Are they watching?" expands into surveillance, observation, and deep philosophical questions on existence and freedom. In a technology-dominant age, this analysis navigates issues of privacy, power dynamics, and the philosophical implications of observation.

**Explorations:**

1. **Surveillance:**
   Examining how modern technology facilitates pervasive surveillance and impacts societal norms.

2. **Philosophical Perspectives:**
   Using Bentham's panopticon to discuss the impact of observation on behavior and freedom.

3. **Privacy and Security:**
   Balancing security enhancement against the right to privacy raises substantial ethical questions.

4. **Existential and Metaphysical Considerations:**
   Discussing the influence of observed existence on morality and purpose.

5. **Technological Impacts:**
   Assessing how surveillance tools necessitate ethical regulations to safeguard freedom.

6. **Behavior Influences:**
   Exploring the chilling effect on personal expression due to constant observation.

**Conclusion:**
"Are they watching?" poses critical societal and philosophical questions about surveillance's role today. It impels us to strike a balance between security measures and safeguarding autonomy. Further investigation in cultural impacts and global surveillance ethics is advocated.

---

By combining insights on the essential role of spiritual practices today and the multifaceted inspection of surveillance, this report provides a groundwork for understanding and addressing these issues. Future societal and policy directions should consider these findings to promote a balanced approach to development, privacy, and human well-being.

# Thank You! 👋🏽

If you liked this notebook, consider following us on GitHub and LinkedIn. If you enjoy high-quality articles on tech, subscribe to our SubStack Newsletter!

**TODO:** Add links here.